In [201]:
!nvidia-smi

Wed Aug 18 16:18:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    30W /  70W |   3160MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **악플 탐지기 모델**

## **파이썬 패키치 설치 및 가져오기**

### **git clone**

In [203]:
# https://github.com/monologg/KoCharELECTRA
!git clone https://github.com/monologg/KoCharELECTRA

# 데이터셋만 가져옴
!git clone https://github.com/Team-M1/preprocessing-for-final-project.git

fatal: destination path 'KoCharELECTRA' already exists and is not an empty directory.
Cloning into 'preprocessing-for-final-project'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 85 (delta 37), reused 52 (delta 6), pack-reused 0
Unpacking objects: 100% (85/85), done.


### **pip install**

In [169]:
!pip install transformers

### **import**

In [170]:
import time
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from torch.utils.data import TensorDataset, DataLoader
from transformers import Trainer, TrainingArguments, ElectraModel

from KoCharELECTRA.tokenization_kocharelectra import KoCharElectraTokenizer

## **기본 설정값**

### **데이터셋**

In [171]:
data_path = "preprocessing-for-final-project/data/혐오표현_정제.csv"

In [172]:
df = pd.read_csv(data_path)
df.head()

,content,hate,gender_hate
0,걍 이 병신나라 민도가 좆박은건데 어쩔수가 잇나 걍 망해야지,1,0
1,그 앰생 쓰레기 병신들은 짐승이고. 니는 짐승을 사람 취급해주냐.,1,0
2,제발 너그집으로 꺼져 애미뒤진새끼야,1,0
3,몸만지면 대주는 걸레년임,1,1
4,환경오염도 이쯤되면 심각하고 자식 낳아봐야 지들 애미애비(그러므로 너 나 우리)가 ...,1,0


In [173]:
X, y = df["content"].to_list(), df["gender_hate"].to_list()

### **토크나이저**

In [174]:
tokenizer = KoCharElectraTokenizer.from_pretrained("monologg/kocharelectra-base-discriminator")
print(tokenizer.tokenize('안녕하세요 제 이름은 김이라고 해요.'))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'KoCharElectraTokenizer'.


['안', '녕', '하', '세', '요', ' ', '제', ' ', '이', '름', '은', ' ', '김', '이', '라', '고', ' ', '해', '요', '.']


### **전이학습할 모델**

In [175]:
pretrained_model = ElectraModel.from_pretrained("monologg/kocharelectra-base-discriminator")

Some weights of the model checkpoint at monologg/kocharelectra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### **기타 (시드 등)**

In [176]:
seed = 1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## **데이터 전처리**

### **Random Over Sampling**

https://imbalanced-learn.org/stable/over_sampling.html

In [177]:
print("gender_hate 0:", y.count(0))
print("gender_hate 1:", y.count(1))

gender_hate 0: 7735
gender_hate 1: 1007


In [178]:
ros = RandomOverSampler(random_state=seed)
X, y = ros.fit_resample(np.asarray(X).reshape(-1, 1), np.asarray(y).reshape(-1, 1))
X, y = list(map(lambda x: x[0], X.tolist())), y.tolist()
print("gender_hate 0:", y.count(0))
print("gender_hate 1:", y.count(1))

gender_hate 0: 7735
gender_hate 1: 7735


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


### **SMOTE**

In [179]:
# https://wyatt37.tistory.com/10

### **Borderline-SMOTE**

In [180]:
# https://datascienceschool.net/03%20machine%20learning/14.02%20%EB%B9%84%EB%8C%80%EC%B9%AD%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%AC%B8%EC%A0%9C.html

### **ADASYN**

In [181]:
# https://wyatt37.tistory.com/10

## **데이터 준비**

### **Text Encoding**

In [182]:
X_encoding = tokenizer(X, padding=True)["input_ids"]
print("Encoding:", np.array(X_encoding).shape)

Encoding: (15470, 172)


### **Train, Val, Test 데이터셋 분리**

In [183]:
X_train, X_test, y_train, y_test = train_test_split(X_encoding, y, test_size=0.1, random_state=seed)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=seed)

In [184]:
print("Training:", np.array(X_train).shape, ",", np.array(y_train).shape)
print("Validation:", np.array(X_val).shape, ",", np.array(y_val).shape)
print("Test:", np.array(X_test).shape, ",", np.array(y_test).shape)

Training: (12530, 172) , (12530,)
Validation: (1393, 172) , (1393,)
Test: (1547, 172) , (1547,)


### **TensorDataset**

In [185]:
train_dataset = TensorDataset(torch.LongTensor(X_train), torch.LongTensor(y_train))
val_dataset = TensorDataset(torch.LongTensor(X_val), torch.LongTensor(y_val))
test_dataset = TensorDataset(torch.LongTensor(X_test), torch.LongTensor(y_test))

### **DataLoader**

In [186]:
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size, num_workers=2, pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, shuffle=True, batch_size=batch_size, num_workers=2, pin_memory = True)
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True, batch_size=batch_size, num_workers=2, pin_memory = True)

## **모델 학습하기**

### **GRU 모델**

In [187]:
import torch.nn as nn

class GRUNet(nn.Module):
    def __init__(self, pretrained, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super().__init__()
        self.pretrained = pretrained
        embedding_dim = pretrained.config.to_dict()['hidden_size']
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [batch size, sent len]

        with torch.no_grad():
            embedded = self.pretrained(text)[0]
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        #output = [batch size, out dim]
        return output

### **학습을 위한 유틸 함수**

In [188]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [189]:
def train(model, dataloader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()

    for data, label in dataloader:
        data = data.cuda()
        label = label.float().cuda()
        optimizer.zero_grad()
        predictions = model(data).squeeze(1)
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [190]:
def evaluate(model, dataloader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for data, label in dataloader:
            data = data.cuda()
            label = label.float().cuda()
            predictions = model(data).squeeze(1)
            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(dataloader), epoch_acc / len(dataloader)

In [191]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### **학습하기**

In [192]:
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2

epochs = 20
best_valid_loss = float('inf')

In [193]:
model = GRUNet(pretrained_model, hidden_dim, output_dim, n_layers, bidirectional, dropout).cuda()

# 전이학습 모델 가중치 고정
for name, param in model.named_parameters():                
    if name.startswith('pretrained'):
        param.requires_grad = False
print("Model parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

Model parameters: 2759169


In [194]:
criterion = nn.BCEWithLogitsLoss().cuda()
optimizer = optim.Adam(model.parameters())

In [195]:
for epoch in range(epochs):
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_loader, criterion)
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 43s
	Train Loss: 0.534 | Train Acc: 73.04%
	 Val. Loss: 0.406 |  Val. Acc: 81.67%
Epoch: 02 | Epoch Time: 2m 55s
	Train Loss: 0.358 | Train Acc: 84.82%
	 Val. Loss: 0.238 |  Val. Acc: 90.77%
Epoch: 03 | Epoch Time: 2m 56s
	Train Loss: 0.237 | Train Acc: 90.57%
	 Val. Loss: 0.215 |  Val. Acc: 92.24%
Epoch: 04 | Epoch Time: 2m 56s
	Train Loss: 0.148 | Train Acc: 94.34%
	 Val. Loss: 0.145 |  Val. Acc: 94.96%
Epoch: 05 | Epoch Time: 2m 56s
	Train Loss: 0.102 | Train Acc: 96.18%
	 Val. Loss: 0.172 |  Val. Acc: 93.85%
Epoch: 06 | Epoch Time: 2m 56s
	Train Loss: 0.078 | Train Acc: 97.17%
	 Val. Loss: 0.118 |  Val. Acc: 96.57%
Epoch: 07 | Epoch Time: 2m 56s
	Train Loss: 0.052 | Train Acc: 98.00%
	 Val. Loss: 0.100 |  Val. Acc: 97.02%
Epoch: 08 | Epoch Time: 2m 55s
	Train Loss: 0.045 | Train Acc: 98.31%
	 Val. Loss: 0.110 |  Val. Acc: 96.71%
Epoch: 09 | Epoch Time: 2m 56s
	Train Loss: 0.040 | Train Acc: 98.54%
	 Val. Loss: 0.126 |  Val. Acc: 96.62%
Epoch: 10 | Epoch T

## **모델 최종 정확도 측정하기**

In [196]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_loader, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.056 | Test Acc: 98.45%


## **데이터 추론하기**

### **kocohub unlabeled 데이터**

In [206]:
!git clone https://github.com/kocohub/korean-hate-speech

Cloning into 'korean-hate-speech'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 112 (delta 4), reused 0 (delta 0), pack-reused 103
Receiving objects: 100% (112/112), 93.18 MiB | 20.89 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Checking out files: 100% (20/20), done.


In [207]:
unlabeled_path = "korean-hate-speech/unlabeled/unlabeled_comments_1.txt"

In [211]:
df_unlabeled = pd.read_csv(unlabeled_path, sep="\n")
df_unlabeled.head()

,지드래곤은 난봉꾼이란...댓글도 달렸네 ㅋㅋ 이주연 학창시절 사진 보고 와라. 요즘 웬만한 여자 연예인하고 붙여놔도....미모가 최고였단다.ㅋ 5대 얼짱 출신.
0,이주연은 알겠는데 지디는 뭐하는 듣보잡여
1,부럽네요. 나도 불과 한달전까진 허니문베이비를 꿈꿨는데 이제 다 부질없네요. 당연히...
2,이주연을 모르는 애들이 많네. 해체된 애프터스쿨 멤버로 당시는 주연이 예명. 인기나...
3,겨론했으면
4,이주연이 아깝다 진심


In [246]:
max_input_length = 172

def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = tokenizer.convert_tokens_to_ids(tokens)
    tensor = torch.LongTensor(indexed).cuda()
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [247]:
predict_sentiment(model, tokenizer, "안녕하세용")


0.01115437038242817

In [250]:
count = 50

with open(unlabeled_path, "r") as f:
  while True:
    if count < 0: break
    count -= 1
    line = f.readline()
    print(line, end="")
    print(predict_sentiment(model, tokenizer, line))
    print("==========")

지드래곤은 난봉꾼이란...댓글도 달렸네 ㅋㅋ 이주연 학창시절 사진 보고 와라. 요즘 웬만한 여자 연예인하고 붙여놔도....미모가 최고였단다.ㅋ 5대 얼짱 출신.
0.9995729327201843
이주연은 알겠는데 지디는 뭐하는 듣보잡여
0.6350446939468384
부럽네요. 나도 불과 한달전까진 허니문베이비를 꿈꿨는데 이제 다 부질없네요. 당연히 순결할거라 믿었고 그래서 첫날밤까지 기다려준건데 배신감만 듭니다. 첫날밤 와이프가 피를 안흘렸어요. 처가집식구들이 일부러 절 속였단 생각에 화도나고 어제 처가집가기로 했는데 안간다고 했더니 혼자 울고 갔다와서 지금까지 한마디도 안해요. 이혼하고 싶네요
0.0005666663637384772
이주연을 모르는 애들이 많네. 해체된 애프터스쿨 멤버로 당시는 주연이 예명. 인기나 포텐은 안터졌으나, 순수미모만으로는 애프터스쿨에서 원탑이었다. 진짜 자연미인이다.
0.00011339018237777054
겨론했으면
0.2412661761045456
이주연이 아깝다 진심
0.000891237985342741
방탄은 건드리지말자 인간적으로.. 해외활동 지금주터 시작인데 터지면 진짜 전 세계적으로 활짝 피기도 전에 난리난다.. 국위선양하는 떠오르는 샛별은 제발 건드리지말고.. 탄이들도 만약 연애하더라도 들키지 않길.. ㅠㅠ
5.655345739796758e-05
선남선녀의 만남이네요^^
0.06127695366740227
문재앙 또뭘덥고 싶어서 ㄷㄷㄷ
0.0007025920785963535
주연이 아깝긴해요
0.00017643717001192272
지디는 아이유랑 너무 잘 어울리는데. 아쉽당~마지막은 아이유랑 해 주세요~ 인류의 축복을 위해서^^
0.001114502432756126
오빠 좀 괴롭지 마요
0.7519452571868896
너희 실수했어 사람 잘못골랐어 이거 오보야
0.0005156744737178087
모두가 거짓기사임..같이 찍은 사진도 없고 지디가 아니라고 분명히 못을 박았음..집근처 사진은 나도 하겠다..구

In [258]:
count = 50

with open(unlabeled_path, "r") as f:
  line = f.readline()
  while True:
    if count < 0: break
    line = f.readline()

    sent = predict_sentiment(model, tokenizer, line)
    if sent > 0.95:
      count -= 1
      print(line, end="")
      print(predict_sentiment(model, tokenizer, line))
      print("==========")

새해부터 이주연 깎아 내리느라 수고한다. 이주연 발톱 때보다 못생긴 오크 쿵쾅녀들아 ㅋㅋㅋ
0.9999630451202393
지디 부럽다 ...모든 20대 여자 다 만날 능력돼서 ㅋㅋㅋ
0.95677649974823
정치, 과학, 사회 관련 기사의 댓글과 관심도 남자95% 여자 5%, 연예관련 기사의 댓글과 관심도 여자 70~90% ㅋㅋㅋㅋㅋㅋㅋ 실화냐? 이러니까 여자들 평균임금과 요직 비율이 낮지 ㅋㅋㅋ 근데 차별핑계 오지게 대더라 ㅋㅋ
0.9991759657859802
딱봐도 엔조이구만 ㅋ 남자색휘들은 잘 알듯~고대로 딱 그만큼 행동하네 ㅎㅎ
0.9999023675918579
주연이가 너무 아까워... 정말 예쁜데... 만날때 힐도 못신고 단화만 신어야될거 아니야... 남자가 170은 안되더라도 최소한 165는 넘어야지 ㅉㅉ...
0.9981776475906372
촛불들던 정신나간 놈
0.992668628692627
척척 찍 응애~
0.9990543723106384
연예계는 동물에왕국맞네 한낮조연 나부랭이가 연기는안하고 연기한답시고 여자나 꼬셔서 지 군대가는 욕구풀이로 한번씩 할려고 여자곰신 만들었네
0.9891170859336853
떡중이라 헤라
0.992137610912323
이아줌마는 뉴규??????????????
0.9996505975723267
준이 잘해?
0.9802688956260681
동물의 왕국에서 또 한쌍의 암수가..
0.9607526659965515
군대에서는 치마만 걸치면 여자 @@
0.9990241527557373
근대 쟤네 누구임?배우임?
0.9500033259391785
동물의 왕국이네 이년아
0.9883915781974792
군인이 휴가 중에 운전을 해도 되나?
0.9993755221366882
안가려도 되는데 오바는
0.997747004032135
여자라는건 그냥 단지 한두번 욕정의 물을 비우기 위한 도구에 지나지 않는데 왜 자꾸들 연애를 할까..? 연애를 해주지 않으면 눕힐수 없으니까? 아님 이 여자만에게 평생 뿌려주고 싶어

### **황도원 데이터**

In [251]:
unlabeled_path2 = "preprocessing-for-final-project/unlabeled_data/concat_unlabeled_data.csv"

In [253]:
count = 50

with open(unlabeled_path2, "r") as f:
  line = f.readline()
  while True:
    if count < 0: break
    count -= 1
    line = f.readline()
    print(line, end="")
    print(predict_sentiment(model, tokenizer, line))
    print("==========")

좌배 까는건 ㅇㅂ,1
0.9953380823135376
집에 롱 패딩만 세 개다. 10년 더 입어야지 ㅋㅋ,0
0.1364128440618515
개소리야 니가 빨갱이를 옹호하고 드루킹을 ㅇㅇ짓이라고 말못해서 삐진거야 빨갱아,1
0.016883691772818565
세탁이라고 봐도 된다,0
0.005189033690840006
애새끼가 초딩도 아니고 ㅋㅋㅋ,1
0.6918653249740601
731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 벌리는 한국에 기를 써서 진출하려고 하지조센남자들은 또 이쁜여자만 보면 사족을 못쓰며 공주대접해주는 놈들이니,1
0.962875485420227
재앙이한건햇노,1
0.9991575479507446
글쓴이 와꾸 승리에 비하면 방사능 피폭 원숭이 일듯..,1
0.06819329410791397
마 씨발련 아 몇평이고 맷개드갔노 니 대하이햄하고 해밨나,1
0.20083019137382507
은행에 대출 상담 받으러 가보면 직업의 귀천 바로 알려줌,0
0.0005461426917463541
ㅋㅋㅋ,0
0.39839810132980347
ㄹㅇㅋㅋㅋ,0
0.11311201751232147
우리지역군데 금태섭 뽑으면 안되지? 그래도 자한당 뻡아야겠지?,0
0.0026321723125874996
꽃다발사들고 알바하는곳찾아가서 무릎꿇고 '내 마음을 받아줄래 지영아?' 하면 바로 넘어온다 여자들 이런거 존나좋아함,1
0.30516284704208374
박근혜 안빠는데 보수통합 3원칙 인정함,1
0.0009512119577266276
대가리에 필터없는 연봉 30억 강사vs대가리가 없는 용접공,1
0.0033032523933798075
백정주제에 라고하는걸 비하라고 느낀다면 본인조차 직업에 자부심이 없는건데,0
0.020350849255919456
이게 다 백인 탓이랑께,0
0.07827646285295486
뭐저런 골빈 ㄴ이 다있냐 ㅋㅋㅋ 저런게 학원강사라고? 애들이 불쌍,1
0.08983074873685837

In [256]:
count = 50

with open(unlabeled_path2, "r") as f:
  line = f.readline()
  while True:
    if count < 0: break
    line = f.readline()

    sent = predict_sentiment(model, tokenizer, line)
    if sent > 0.95:
      count -= 1
      print(line, end="")
      print(predict_sentiment(model, tokenizer, line))
      print("==========")

좌배 까는건 ㅇㅂ,1
0.9953380823135376
731부대의 후예라 그런지 가학적인 아이디어는 세계최고임 이래서 애교만 떨어도 돈 벌리는 한국에 기를 써서 진출하려고 하지조센남자들은 또 이쁜여자만 보면 사족을 못쓰며 공주대접해주는 놈들이니,1
0.962875485420227
재앙이한건햇노,1
0.9991575479507446
ㅋㅋ ㅇㅈ 저런거에 앉아어 감전사 당할 수도,0
0.9796949625015259
군대에서 여군이 필요한가,0
0.992216944694519
사형 예지!,0
0.999182403087616
흑종원 ㅋㅋㅋ,0
0.9727607369422913
호주에서는 남자들이 동물보다 못한존재라던데ㅋㅋ,0
0.9981912970542908
너도 해,0
0.9686318039894104
니들 틀딱들 닭빨고 진빨고 중국빨고 그때그때 빠는새끼들임 닭이 친중하면 중국도 빨고 오성홍기절하고 아님? 억울하면 논리적으로 반박을 해 ㅋ,1
0.9780762791633606
그냥 홀딩하세요,0
0.9805361032485962
오마쥬가 뭔진 앎?,0
0.9797841310501099
쓰레기들은 휴지통으로,0
0.9589219689369202
앞머리 좀 자르던가 ㅅㅂ,1
0.9995286464691162
개찐따 답네 ㅋㅋㅋ,1
0.999032735824585
오이 오이 아부네,0
0.9989733695983887
이봐요 미친놈씨,1
0.999839186668396
이게 여자다,0
0.9976119995117188
키 170 27살 따먹은여자 23명 ㅁㅌㅊ?,1
0.999806821346283
극혐이노 ㅅㅂ ㅋㅋㅋ,1
0.9958179593086243
공지이잉 ㅅㅂㅋㅋㅋ,1
0.9931545257568359
ㅋㅋㅋ ㄹㅇ,0
0.9909341335296631
전라남도 안정권 홍어새끼 좆이나 빠는새끼가 가세연을 까노 ㅋㅋㅋ,1
0.9654722809791565
강사도 누구나 함 주둥이만있으면,1
0.9699535369873047
운지햐라,1
0.9862536191940